In [1]:
import pandas as pd
import numpy as np

In [2]:
# Data Augmentation

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
train_gen = ImageDataGenerator(rescale=(1./255))
test_gen = ImageDataGenerator(rescale=(1./255))  #--> (0 to 255) convert to (0 to 1)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!unzip "/content/drive/MyDrive/Dataset.zip" -d "/content/Dataset/"

Archive:  /content/drive/MyDrive/Dataset.zip
   creating: /content/Dataset/Dataset/test/
   creating: /content/Dataset/Dataset/test/aloperia areata/
  inflating: /content/Dataset/Dataset/test/aloperia areata/1.PNG  
 extracting: /content/Dataset/Dataset/test/aloperia areata/12.PNG  
  inflating: /content/Dataset/Dataset/test/aloperia areata/13.PNG  
 extracting: /content/Dataset/Dataset/test/aloperia areata/14.PNG  
  inflating: /content/Dataset/Dataset/test/aloperia areata/2.PNG  
  inflating: /content/Dataset/Dataset/test/aloperia areata/23.PNG  
  inflating: /content/Dataset/Dataset/test/aloperia areata/24.PNG  
 extracting: /content/Dataset/Dataset/test/aloperia areata/25.PNG  
  inflating: /content/Dataset/Dataset/test/aloperia areata/3.PNG  
  inflating: /content/Dataset/Dataset/test/aloperia areata/34.PNG  
 extracting: /content/Dataset/Dataset/test/aloperia areata/35.PNG  
  inflating: /content/Dataset/Dataset/test/aloperia areata/36.PNG  
  inflating: /content/Dataset/Dataset/

In [4]:
train = train_gen.flow_from_directory('/content/Dataset/Dataset/train/',
                                      target_size=(224, 224),
                                      class_mode='categorical',
                                      batch_size=8)
test = test_gen.flow_from_directory('/content/Dataset/Dataset/test/',
                                    target_size=(224, 224),
                                      class_mode='categorical',
                                      batch_size=8)

Found 656 images belonging to 17 classes.
Found 204 images belonging to 17 classes.


In [5]:
train.class_indices

{'Darier_s disease': 0,
 'Healthy nail': 1,
 'Muehrck-e_s lines': 2,
 'aloperia areata': 3,
 'beau_s lines': 4,
 'bluish nail': 5,
 'clubbing': 6,
 'eczema': 7,
 'half and half nailes (Lindsay_s nails)': 8,
 'koilonychia': 9,
 'leukonychia': 10,
 'onycholycis': 11,
 'pale nail': 12,
 'red lunula': 13,
 'splinter hemmorrage': 14,
 'terry_s nail': 15,
 'yellow nails': 16}

In [6]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121, VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping

In [7]:

# Set hyperparameters and configurations
num_classes = 17
batch_size = 32
learning_rate = 0.001
num_epochs = 30

In [8]:
# Define your DenseNet-VGG16 hybrid model architecture
def DenseNetVGG16(num_classes):
    # Load pre-trained DenseNet and VGG16 models
    densenet = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Freeze the pre-trained layers
    for layer in densenet.layers:
        layer.trainable = False
    for layer in vgg16.layers:
        layer.trainable = False

    # Combine DenseNet and VGG16 models
    input_layer = tf.keras.Input(shape=(224, 224, 3))
    densenet_output = densenet(input_layer)
    vgg16_output = vgg16(input_layer)

    # Add global average pooling layer
    densenet_output = GlobalAveragePooling2D()(densenet_output)
    vgg16_output = GlobalAveragePooling2D()(vgg16_output)

    # Concatenate DenseNet and VGG16 outputs
    combined_output = tf.keras.layers.concatenate([densenet_output, vgg16_output])

    # Add a fully connected layer
    dense = Dense(units=128, activation='relu')(combined_output)

    # Add output layer
    output = Dense(units=num_classes, activation='softmax')(dense)

    # Create the model
    model = Model(inputs=input_layer, outputs=output)

    return model

In [9]:
# Load and preprocess the nail image dataset
# Ensure that you have a data generator providing batches of image-label pairs
# Initialize your DenseNet-VGG16 hybrid model
model = DenseNetVGG16(num_classes)

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 densenet121 (Functional)    (None, 7, 7, 1024)           7037504   ['input_3[0][0]']             
                                                                                                  
 vgg16 (Functional)          (None, 7, 7, 512)            1471468   ['input_3[0][0]']             
                                                          8                                       
                                                                                                  
 global_average_pooling2d (  (None, 1024)                 0         ['densenet121[0][0]']     

In [11]:
# Create the optimizer with a specific learning rate
optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)

# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='accuracy', patience=3, restore_best_weights=True)

In [13]:
# Train the model
history = model.fit(train,epochs=num_epochs,validation_data=test, callbacks=[early_stopping])

Epoch 1/30
82/82 [==============================] - 25s 165ms/step - loss: 2.7998 - accuracy: 0.1387 - val_loss: 2.2817 - val_accuracy: 0.3627
Epoch 2/30
82/82 [==============================] - 9s 111ms/step - loss: 2.2098 - accuracy: 0.3034 - val_loss: 2.1732 - val_accuracy: 0.2990
Epoch 3/30
82/82 [==============================] - 10s 117ms/step - loss: 1.7843 - accuracy: 0.4497 - val_loss: 1.3456 - val_accuracy: 0.6814
Epoch 4/30
82/82 [==============================] - 10s 118ms/step - loss: 1.4903 - accuracy: 0.5305 - val_loss: 1.2005 - val_accuracy: 0.6225
Epoch 5/30
82/82 [==============================] - 10s 117ms/step - loss: 1.2318 - accuracy: 0.6280 - val_loss: 1.0294 - val_accuracy: 0.7157
Epoch 6/30
82/82 [==============================] - 10s 120ms/step - loss: 1.0050 - accuracy: 0.7073 - val_loss: 0.8408 - val_accuracy: 0.7990
Epoch 7/30
82/82 [==============================] - 9s 111ms/step - loss: 0.8587 - accuracy: 0.7698 - val_loss: 0.7048 - val_accuracy: 0.8235
E

In [14]:
test_images, true_labels = [], []
for _ in range(len(test)):
    images, labels = test.next()
    test_images.append(images)
    true_labels.append(labels)


In [15]:
test_images = np.concatenate(test_images)
true_labels = np.argmax(np.concatenate(true_labels), axis=1)

In [16]:
# Make predictions on the test set
predictions = model.predict(test_images)

7/7 [==============================] - 12s 670ms/step


In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [18]:
# Convert predictions to labels
predicted_labels = np.argmax(predictions, axis=1)

In [19]:
accuracy = accuracy_score(true_labels, predicted_labels)
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.9461


In [20]:
# Precision
precision = precision_score(true_labels, predicted_labels, average='weighted')
print(f'Precision: {precision:.4f}')

Precision: 0.9570


In [21]:
# Recall
recall = recall_score(true_labels, predicted_labels, average='weighted')
print(f'Recall: {recall:.4f}')

Recall: 0.9461


In [22]:
# F1 score
f1 = f1_score(true_labels, predicted_labels, average='weighted')
print(f'F1 Score: {f1:.4f}')


F1 Score: 0.9435


In [23]:
# Confusion Matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print('Confusion Matrix:')
print(conf_matrix)

Confusion Matrix:
[[17  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 17  1  0  0  0  0  2  0  1  2  0  0  0  1  0  3]
 [ 0  0  9  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 15  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  8  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 13  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 12  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 12  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 15  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  8  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  6  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  0  0  0 11  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  8  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0 15  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0 10  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  9  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  8]]


In [24]:
# Save the trained model
model.save('DenseNetVGG16.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [25]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [26]:
model = load_model('DenseNetVGG16.h5')

In [57]:
img=image.load_img('/content/Dataset/Dataset/test/yellow nails/1.PNG',target_size=(224,224))

In [58]:
img = image.img_to_array(img)
img = np.expand_dims(img,axis=0)
model.predict(img)
pred = np.argmax(model.predict(img))-1
output = ['Darier_s disease','Healthy Nail','Muehrck-e_s lines','Aloperia areata',
          'beau_s lines','bluish nail', 'clubbing','eczema','half and half nailes (Lindsay_s nails)',
          'koilonychia','leukonychia','pale nail','red lunula',
          'splinter hemmorrage_Acral Lentiginous Melanoma',
          'terry_s nail_WhiteNails','yellow nails']
print(output[pred])

1/1 [==============================] - 0s 47ms/step
yellow nails
